# Importing

In [ ]:
# # keep runtime
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# var clicker = setInterval(ClickConnect,60000);
# # stop keeping runtime 
# clearInterval(clicker);

from google.colab import drive
drive.mount('/content/drive')

!pip install --upgrade scikit-learn
!pip install --upgrade statsmodels
!pip install --upgrade xgboost

# # lightgbm gpu version
# !pip uninstall lightgbmt --yes
# !pip install --upgrade cmake
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd LightGBM
# !mkdir build
# %cd build
# !cmake -DUSE_GPU=1 ..
# !make -j$(nproc)
# %cd ../python-package
# !python3 setup.py install --gpu

!pip install --upgrade catboost
!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install --upgrade keras-tuner
!pip install --upgrade tensorflow-addons
!pip install --upgrade optuna
!pip install --upgrade wandb

import os
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/projects/DA_Platform")
from DA_v5 import *
import wandb
from wandb.keras import WandbCallback
from optuna.integration.wandb import WeightsAndBiasesCallback as optuna_wnbcallback
from optuna import pruners
import shutil
from glob import glob
from IPython.display import Image, display

import sys
from multiprocessing import cpu_count
import copy
import pickle
import warnings
from datetime import datetime, timedelta
from time import time, sleep, mktime
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import random
import wandb
from wandb.keras import WandbCallback
import shutil

import numpy as np
from numpy import array, nan, random as rnd, where
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset
import statsmodels.api as sm
from scipy.stats import f_oneway

from sklearn import preprocessing as prep
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split as tts, GridSearchCV as GridTuner, StratifiedKFold, KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn import metrics
from sklearn.pipeline import make_pipeline

from sklearn import linear_model as lm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn import neighbors as knn
from sklearn import ensemble
from optuna import distributions as optuna_dist, visualization as optuna_plt, Trial, create_study
from optuna.integration import OptunaSearchCV
from optuna.samplers import TPESampler
from optuna.logging import set_verbosity as optuna_set_verbose

# ===== tensorflow =====
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import metrics as tf_metrics
from tensorflow.keras import callbacks as tf_callbacks
from tqdm.keras import TqdmCallback
import tensorflow_addons as tfa
import keras_tuner as kt
from keras_tuner import HyperModel
from tensorflow.keras.utils import plot_model

# ===== timeseries =====
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing import timeseries_dataset_from_array as make_ts_tensor

# global setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)

rcParams['axes.unicode_minus'] = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["WANDB_API_KEY"] = "6f810b088fcc6b9eaaa56c1e52cfd37836606240"
folder_path = "/content/drive/MyDrive/Colab Notebooks/projects/Dacon/jobknow_recommendation/"

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

Mounted at /content/drive
     |████████████████████████████████| 9.8 MB 7.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 173.5 MB 10 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
     |████████████████████████████████| 76.1 MB 107 kB/s 
     |████████████████████████████████| 98 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 6.8 MB/s 
     |████████████████████████████████| 308 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 72.4 MB/s 
     |████████████████████████████████| 80 kB 11.7 MB/s 
     |████████████████████████████████| 75 kB 6.2 MB/s 
     |████████████████████████████████| 149 kB 97.0 MB/s 
     |████████████████████████████████| 112 kB 85.2 MB/s 
     |██████████

# Preprocessing - 2017

In [ ]:
# # ===== Preprocessing =====
# target_year = "2017"
# target_var = "knowcode"
# full_x = read_csv(folder_path + "train/KNOW_" + target_year + ".csv", dtype={target_var: str}, na_values=["", " "], encoding="utf-8")
# full_x.drop(["idx"], axis=1, inplace=True)
# print("before removing dups", full_x.shape[0])
# full_x.drop_duplicates(inplace=True)
# print("after removing dups", full_x.shape[0])

# print("before removing invalid values on target", full_x.shape[0])
# full_x[target_var].replace(" ", "", inplace=True)
# full_x = full_x[(full_x[target_var] != "9999999") & (full_x[target_var] != nan)]
# print("after removing invalid values on target", full_x.shape[0])
# full_x.reset_index(drop=True, inplace=True)

# full_y = full_x[target_var].to_frame()
# target_encoder = copy.deepcopy(MyLabelEncoder())
# full_y = target_encoder.fit_transform(full_y, [target_var])
# print(target_encoder.dic_cat)
# full_y.isna().sum()
# full_y[target_var] = full_y[target_var].astype("int")
# print(full_y[target_var].value_counts())
# full_x.drop([target_var], axis=1, inplace=True)

# test_x = read_csv(folder_path + "test/KNOW_" + target_year + "_test.csv", dtype={target_var: str}, na_values=["", " "], encoding="utf-8")
# submission_idx_start = test_x["idx"].iloc[0]
# submission_idx_end = test_x["idx"][-1:].iloc[0]
# test_x.drop(["idx"], axis=1, inplace=True)



# # # text feature 정리
# # "bq4_1a" 필요자격증1
# # "bq4_1b" 필요자격증2
# # "bq4_1c" 필요자격증3
# # "bq5_2" 요구훈련종류
# # "bq19_1" : 향후 일자리 변화 이유
# # "bq30" : 유사직업명
# # "bq31" : 사용도구 및 프로그램
# # "bq32" : 이전직업
# # "bq33" : 전직 가능 직업
# # "bq34" : 신직업 발굴
# # "bq38_1" : 전공

# # "bq31" 사용도구 및 프로그램은 콤마로 구분되는 멀티레이블 컬럼으로 볃도로 처리
# text_vars = ["bq4_1a", "bq4_1b", "bq4_1c", "bq5_2", "bq19_1",
#              "bq30", "bq31_0", "bq31_1", "bq31_2", "bq32", "bq33", "bq34", "bq38_1"]

# # "bq31" 사용도구 및 프로그램 -> 세 컬럼 분리
# max_size_tool = 3
# bq31_split_vector = full_x["bq31"].str.split(",")
# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = bq31_split_vector.str[i]

# bq31_split_vector = test_x["bq31"].str.split(",")
# for i in range(max_size_tool):
#     test_x["bq31_" + str(i)] = bq31_split_vector.str[i]

# # text 컬럼 전역 전처리
# for i in text_vars:
#     for idx, value in enumerate(full_x[i]):
#         if isna(value):
#             full_x[i][idx] = "none"
#         else:
#             value = text_extractor(value, "both", False).lower()
#             full_x[i][idx] = value

# for i in text_vars:
#     for idx, value in enumerate(test_x[i]):
#         if isna(value):
#             test_x[i][idx] = "none"
#         else:
#             value = text_extractor(value, "both", False).lower()
#             test_x[i][idx] = value


# # "bq4_1a" 필요자격증
# # 기술사 - 기능장 - 기사 - 산업기사 - 기능사
# remove_text = ["기술사", "기능장", "기사", "산업기사", "기능사",
#                "자격증", "자격", "면허증", "면허", "전문가"]
# for idx, value in enumerate(full_x["bq4_1a"]):
#     for i in remove_text:
#         value = value.replace(i, "")
#     full_x["bq4_1a"][idx] = value

# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() > i]) < 100:
#         high_freq = full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() > i]
#         break
# # high_freq = full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() > 9]
# print(len(high_freq))

# for idx, value in enumerate(test_x["bq4_1a"]):
#     for i in remove_text:
#         value = value.replace(i, "")
#     test_x["bq4_1a"][idx] = value

# # remove low frequency
# full_x["bq4_1a"] = full_x["bq4_1a"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq4_1a"] = test_x["bq4_1a"].apply(lambda x: x if x in high_freq else "none")
# # print(full_x["bq4_1a"].value_counts())

# # "bq5_2" 요구훈련 종류
# # print(full_x["bq5_2"].value_counts())
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq5_2"].value_counts().index[full_x["bq5_2"].value_counts() > i]) < 100:
#         high_freq = full_x["bq5_2"].value_counts().index[full_x["bq5_2"].value_counts() > i]
#         break
# # high_freq = full_x["bq5_2"].value_counts().index[full_x["bq5_2"].value_counts() > 9]
# print(len(high_freq))

# # remove low frequency
# full_x["bq5_2"] = full_x["bq5_2"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq5_2"] = test_x["bq5_2"].apply(lambda x: x if x in high_freq else "none")
# # print(full_x["bq5_2"].value_counts())


# # "bq30" 유사직업명
# print(full_x["bq30"].value_counts())
# full_x["bq30"] = full_x["bq30"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq30"].value_counts().index[full_x["bq30"].value_counts() > i]) < 100:
#         high_freq = full_x["bq30"].value_counts().index[full_x["bq30"].value_counts() > i]
#         break
# # high_freq = full_x["bq30"].value_counts().index[full_x["bq30"].value_counts() > 9]
# print(len(high_freq))
# test_x["bq30"] = test_x["bq30"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq30"] = full_x["bq30"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq30"] = test_x["bq30"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq30"].value_counts())



# # "bq31" 사용도구 및 프로그램 -> "bq31_0", "bq31_1", "bq31_2"
# bq31_vars = ["bq31_0", "bq31_1", "bq31_2"]
# max_size_tool = 3
# tmp = series()
# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = full_x["bq31_" + str(i)].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
#     tmp = tmp.append(full_x["bq31_" + str(i)])
# tmp.reset_index(drop=True, inplace=True)
# high_freq = None
# for i in range(1, 100):
#     if len(tmp.value_counts().index[tmp.value_counts() > i]) < 100:
#         high_freq = tmp.value_counts().index[tmp.value_counts() > i]
#         break
# # high_freq = tmp.value_counts().index[tmp.value_counts() > 2]
# print(len(high_freq))
# for i in range(max_size_tool):
#     test_x["bq31_" + str(i)] = test_x["bq31_" + str(i)].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = full_x["bq31_" + str(i)].apply(lambda x: x if x in high_freq else "none")
#     test_x["bq31_" + str(i)] = test_x["bq31_" + str(i)].apply(lambda x: x if x in high_freq else "none")

# # transform to BOW
# from sklearn.feature_extraction.text import CountVectorizer
# bq31_bow_encoder = CountVectorizer()

# tmp = []
# for i in zip(full_x[bq31_vars[0]], full_x[bq31_vars[1]], full_x[bq31_vars[2]]):
#     tmp.append(" ".join(i))
# bow = bq31_bow_encoder.fit_transform(tmp)
# full_x = pd.concat([full_x, dataframe(bow.toarray(), columns=["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))])], axis=1)

# tmp = []
# for i in zip(test_x[bq31_vars[0]], test_x[bq31_vars[1]], test_x[bq31_vars[2]]):
#     tmp.append(" ".join(i))
# bow = bq31_bow_encoder.transform(tmp)
# test_x = pd.concat([test_x, dataframe(bow.toarray(), columns=["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))])], axis=1)


# # "bq32" 이전 직업
# print(full_x["bq32"].value_counts())
# full_x["bq32"] = full_x["bq32"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq32"].value_counts().index[full_x["bq32"].value_counts() > i]) < 100:
#         high_freq = full_x["bq32"].value_counts().index[full_x["bq32"].value_counts() > i]
#         break
# # high_freq = full_x["bq32"].value_counts().index[full_x["bq32"].value_counts() > 2]
# print(len(high_freq))
# test_x["bq32"] = test_x["bq32"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq32"] = full_x["bq32"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq32"] = test_x["bq32"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq32"].value_counts())


# # "bq33" 전직가능직업
# print(full_x["bq33"].value_counts())
# full_x["bq33"] = full_x["bq33"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq33"].value_counts().index[full_x["bq33"].value_counts() > i]) < 100:
#         high_freq = full_x["bq33"].value_counts().index[full_x["bq33"].value_counts() > i]
#         break
# # high_freq = full_x["bq33"].value_counts().index[full_x["bq33"].value_counts() > 2]
# print(len(high_freq))
# test_x["bq33"] = test_x["bq33"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq33"] = full_x["bq33"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq33"] = test_x["bq33"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq33"].value_counts())


# # "bq34" 신직업 발굴
# full_x["bq34"] = full_x["bq34"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq34"].value_counts().index[full_x["bq34"].value_counts() > i]) < 100:
#         high_freq = full_x["bq34"].value_counts().index[full_x["bq34"].value_counts() > i]
#         break
# # high_freq = full_x["bq34"].value_counts().index[full_x["bq34"].value_counts() > 2]
# len(high_freq)
# test_x["bq34"] = test_x["bq34"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq34"] = full_x["bq34"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq34"] = test_x["bq34"].apply(lambda x: x if x in high_freq else "none")

# # "bq38_1" 전공
# print(full_x["bq38_1"].value_counts())
# full_x["bq38_1"] = full_x["bq38_1"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq38_1"].value_counts().index[full_x["bq38_1"].value_counts() > i]) < 100:
#         high_freq = full_x["bq38_1"].value_counts().index[full_x["bq38_1"].value_counts() > i]
#         break
# # high_freq = full_x["bq38_1"].value_counts().index[full_x["bq38_1"].value_counts() > 2]
# print(len(high_freq))
# test_x["bq38_1"] = test_x["bq38_1"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq38_1"] = full_x["bq38_1"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq38_1"] = test_x["bq38_1"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq38_1"].value_counts())

# full_x.isna().sum()
# print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# test_x.isna().sum()
# print(test_x.isna().sum().index[full_x.isna().sum() != 0])




# # age transformation
# full_x["bq37_age_cat"] = pd.cut(full_x["bq37"], [i * 10 for i in range(1, 10)], right=False).astype("object")
# test_x["bq37_age_cat"] = pd.cut(test_x["bq37"], [i * 10 for i in range(1, 10)], right=False).astype("object")

# # === 불필요 컬럼 drop 및 이상치 제거 ===
# # 해당 업계 초봉 drop
# full_x.drop(["bq41_2"], axis=1, inplace=True)
# test_x.drop(["bq41_2"], axis=1, inplace=True)

# # 불필요 텍스트 컬럼 drop
# full_x.drop(["bq4_1b", "bq4_1c", "bq19_1", "bq31", "bq31_0", "bq31_1", "bq31_2"], axis=1, inplace=True)
# test_x.drop(["bq4_1b", "bq4_1c", "bq19_1", "bq31", "bq31_0", "bq31_1", "bq31_2"], axis=1, inplace=True)
# text_vars = diff(text_vars, ["bq4_1b", "bq4_1c", "bq19_1", "bq31", "bq31_0", "bq31_1", "bq31_2"])

# # 이상치 aq41_1 == 33
# full_y = full_y[~(full_x["aq41_1"] == 33)]
# full_x = full_x[~(full_x["aq41_1"] == 33)]
# full_x.reset_index(drop=True, inplace=True)
# full_y.reset_index(drop=True, inplace=True)

# full_x["bq41_1"].fillna(0.0, inplace=True)
# full_x["bq41_3"].fillna(0.0, inplace=True)
# full_x["bq41_1_unemp"] = [1 if i == 0 else 0 for i in full_x["bq41_1"]]
# full_x["bq41_3_unemp"] = [1 if i == 0 else 0 for i in full_x["bq41_3"]]

# test_x["bq41_1"].fillna(0.0, inplace=True)
# test_x["bq41_3"].fillna(0.0, inplace=True)
# test_x["bq41_1_unemp"] = [1 if i == 0 else 0 for i in test_x["bq41_1"]]
# test_x["bq41_3_unemp"] = [1 if i == 0 else 0 for i in test_x["bq41_3"]]

# bin_vars = []
# ord_vars = list(full_x.columns[:82])
# # ord_vars = []
# text_vars += ["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))]
# num_vars = ["bq23", "bq37", "bq41_1", "bq41_3"]


# full_x.isna().sum()
# print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# test_x.isna().sum()
# print(test_x.isna().sum().index[full_x.isna().sum() != 0])


# full_x[diff(full_x.columns, text_vars)] = full_x[diff(full_x.columns, text_vars)].fillna(0.0)
# test_x[diff(full_x.columns, text_vars)] = test_x[diff(full_x.columns, text_vars)].fillna(0.0)

# text_vars = ["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))]
# cat_vars = diff(full_x.columns, num_vars + ord_vars + bin_vars + text_vars)

# # columns length check
# print("columns lenght check")
# print(len(full_x.columns) == len(num_vars) + len(ord_vars) + len(bin_vars) + len(cat_vars) + len(text_vars))



# # full_x.isna().sum()
# # print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# # test_x.isna().sum()
# # print(test_x.isna().sum().index[full_x.isna().sum() != 0])


# label_encoder = copy.deepcopy(MyLabelEncoder())
# full_x = label_encoder.fit_transform(full_x, cat_vars + bin_vars + ord_vars)
# for i in cat_vars:
#     print(full_x[i].value_counts().sort_index(), "\n")
# test_x = label_encoder.transform(test_x)
# for i in cat_vars:
#     print(test_x[i].value_counts().sort_index(), "\n")


# # string var check
# for i in full_x:
#     if full_x[i].dtype == "object":
#         print(i)

# # print(full_x.isna().sum())
# # print(full_x.isna().sum().sum())
# # print(test_x.isna().sum())
# # print(test_x.isna().sum().sum())

# knn_imputer = copy.deepcopy(MyKNNImputer())
# full_x = knn_imputer.fit_transform(full_x, full_y, cat_vars + bin_vars + ord_vars + text_vars)
# test_x = knn_imputer.transform(test_x)

# print(full_x.isna().sum().sum())
# print(test_x.isna().sum().sum())

# # string var check
# for i in full_x:
#     if full_x[i].dtype == "object":
#         print(i)

# # === one hot encoding ===
# # print(full_x.isna().sum().sum())
# # print(test_x.isna().sum().sum())

# # print(full_x.shape[1] == len(cat_vars) + len(bin_vars) + len(ord_vars) + len(num_vars))

# # print(full_x.head(20))
# # print(test_x.head(20))

# oh_encoder = copy.deepcopy(MyOneHotEncoder())
# full_x_oh = oh_encoder.fit_transform(full_x, cat_vars)
# test_x_oh = oh_encoder.transform(test_x)
# categoIdx = findIdx(full_x.columns, cat_vars)

# # print(full_x_oh.isna().sum().sum())
# # print(test_x_oh.isna().sum().sum())

# print(full_x.shape)
# print(test_x.shape)
# print(full_x_oh.shape)
# print(test_x_oh.shape)

before removing dups 9486
after removing dups 9486
before removing invalid values on target 9486
after removing invalid values on target 9104
{'knowcode': {'110101': 0, '110102': 1, '110103': 2, '110104': 3, '110105': 4, '110201': 5, '110202': 6, '110203': 7, '11101': 8, '11102': 9, '11201': 10, '12101': 11, '12102': 12, '121101': 13, '121102': 14, '121103': 15, '121104': 16, '121105': 17, '121201': 18, '12201': 19, '122101': 20, '122102': 21, '122103': 22, '122104': 23, '122105': 24, '122201': 25, '122301': 26, '12301': 27, '12401': 28, '12402': 29, '13101': 30, '131101': 31, '131201': 32, '131202': 33, '131203': 34, '132002': 35, '132004': 36, '13201': 37, '13202': 38, '13203': 39, '13204': 40, '13305': 41, '133101': 42, '133201': 43, '133202': 44, '133203': 45, '133204': 46, '133205': 47, '133301': 48, '133302': 49, '13401': 50, '134101': 51, '134102': 52, '134201': 53, '134301': 54, '134302': 55, '134303': 56, '134401': 57, '135001': 58, '13501': 59, '136001': 60, '136002': 61, '13

# Preprocessing - 2017 (embedding layer)

In [ ]:
# # ===== Preprocessing =====
# target_year = "2017"
# target_var = "knowcode"
# full_x = read_csv(folder_path + "train/KNOW_" + target_year + ".csv", dtype={target_var: str}, na_values=["", " "], encoding="utf-8")
# full_x.drop(["idx"], axis=1, inplace=True)
# print("before removing dups", full_x.shape[0])
# full_x.drop_duplicates(inplace=True)
# print("after removing dups", full_x.shape[0])

# print("before removing invalid values on target", full_x.shape[0])
# full_x[target_var].replace(" ", "", inplace=True)
# full_x = full_x[(full_x[target_var] != "9999999") & (full_x[target_var] != nan)]
# print("after removing invalid values on target", full_x.shape[0])
# full_x.reset_index(drop=True, inplace=True)

# full_y = full_x[target_var].to_frame()
# target_encoder = copy.deepcopy(MyLabelEncoder())
# full_y = target_encoder.fit_transform(full_y, [target_var])
# print(target_encoder.dic_cat)
# full_y.isna().sum()
# full_y[target_var] = full_y[target_var].astype("int")
# print(full_y[target_var].value_counts())
# full_x.drop([target_var], axis=1, inplace=True)

# test_x = read_csv(folder_path + "test/KNOW_" + target_year + "_test.csv", dtype={target_var: str}, na_values=["", " "], encoding="utf-8")
# submission_idx_start = test_x["idx"].iloc[0]
# submission_idx_end = test_x["idx"][-1:].iloc[0]
# test_x.drop(["idx"], axis=1, inplace=True)



# # # text feature 정리
# # "bq4_1a" 필요자격증1
# # "bq4_1b" 필요자격증2
# # "bq4_1c" 필요자격증3
# # "bq5_2" 요구훈련종류
# # "bq19_1" : 향후 일자리 변화 이유
# # "bq30" : 유사직업명
# # "bq31" : 사용도구 및 프로그램
# # "bq32" : 이전직업
# # "bq33" : 전직 가능 직업
# # "bq34" : 신직업 발굴
# # "bq38_1" : 전공

# # "bq31" 사용도구 및 프로그램은 콤마로 구분되는 멀티레이블 컬럼으로 볃도로 처리
# text_vars = ["bq4_1a", "bq4_1b", "bq4_1c", "bq5_2", "bq19_1",
#              "bq30", "bq31_0", "bq31_1", "bq31_2", "bq32", "bq33", "bq34", "bq38_1"]

# # "bq31" 사용도구 및 프로그램 -> 세 컬럼 분리
# max_size_tool = 3
# bq31_split_vector = full_x["bq31"].str.split(",")
# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = bq31_split_vector.str[i]

# bq31_split_vector = test_x["bq31"].str.split(",")
# for i in range(max_size_tool):
#     test_x["bq31_" + str(i)] = bq31_split_vector.str[i]

# # text 컬럼 전역 전처리
# for i in text_vars:
#     for idx, value in enumerate(full_x[i]):
#         if isna(value):
#             full_x[i][idx] = "none"
#         else:
#             value = text_extractor(value, "both", False).lower()
#             full_x[i][idx] = value

# for i in text_vars:
#     for idx, value in enumerate(test_x[i]):
#         if isna(value):
#             test_x[i][idx] = "none"
#         else:
#             value = text_extractor(value, "both", False).lower()
#             test_x[i][idx] = value


# # "bq4_1a" 필요자격증
# # 기술사 - 기능장 - 기사 - 산업기사 - 기능사
# remove_text = ["기술사", "기능장", "기사", "산업기사", "기능사",
#                "자격증", "자격", "면허증", "면허", "전문가"]
# for idx, value in enumerate(full_x["bq4_1a"]):
#     for i in remove_text:
#         value = value.replace(i, "")
#     full_x["bq4_1a"][idx] = value

# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() > i]) < 100:
#         high_freq = full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() > i]
#         break
# # high_freq = full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() > 9]
# print(len(high_freq))

# for idx, value in enumerate(test_x["bq4_1a"]):
#     for i in remove_text:
#         value = value.replace(i, "")
#     test_x["bq4_1a"][idx] = value

# # remove low frequency
# full_x["bq4_1a"] = full_x["bq4_1a"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq4_1a"] = test_x["bq4_1a"].apply(lambda x: x if x in high_freq else "none")
# # print(full_x["bq4_1a"].value_counts())

# # "bq5_2" 요구훈련 종류
# # print(full_x["bq5_2"].value_counts())
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq5_2"].value_counts().index[full_x["bq5_2"].value_counts() > i]) < 100:
#         high_freq = full_x["bq5_2"].value_counts().index[full_x["bq5_2"].value_counts() > i]
#         break
# # high_freq = full_x["bq5_2"].value_counts().index[full_x["bq5_2"].value_counts() > 9]
# print(len(high_freq))

# # remove low frequency
# full_x["bq5_2"] = full_x["bq5_2"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq5_2"] = test_x["bq5_2"].apply(lambda x: x if x in high_freq else "none")
# # print(full_x["bq5_2"].value_counts())


# # "bq30" 유사직업명
# print(full_x["bq30"].value_counts())
# full_x["bq30"] = full_x["bq30"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq30"].value_counts().index[full_x["bq30"].value_counts() > i]) < 100:
#         high_freq = full_x["bq30"].value_counts().index[full_x["bq30"].value_counts() > i]
#         break
# # high_freq = full_x["bq30"].value_counts().index[full_x["bq30"].value_counts() > 9]
# print(len(high_freq))
# test_x["bq30"] = test_x["bq30"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq30"] = full_x["bq30"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq30"] = test_x["bq30"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq30"].value_counts())



# # "bq31" 사용도구 및 프로그램 -> "bq31_0", "bq31_1", "bq31_2"
# bq31_vars = ["bq31_0", "bq31_1", "bq31_2"]
# max_size_tool = 3
# tmp = series()
# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = full_x["bq31_" + str(i)].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
#     tmp = tmp.append(full_x["bq31_" + str(i)])
# tmp.reset_index(drop=True, inplace=True)
# high_freq = None
# for i in range(1, 100):
#     if len(tmp.value_counts().index[tmp.value_counts() > i]) < 100:
#         high_freq = tmp.value_counts().index[tmp.value_counts() > i]
#         break
# # high_freq = tmp.value_counts().index[tmp.value_counts() > 2]
# print(len(high_freq))
# for i in range(max_size_tool):
#     test_x["bq31_" + str(i)] = test_x["bq31_" + str(i)].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = full_x["bq31_" + str(i)].apply(lambda x: x if x in high_freq else "none")
#     test_x["bq31_" + str(i)] = test_x["bq31_" + str(i)].apply(lambda x: x if x in high_freq else "none")

# # # transform to BOW
# # from sklearn.feature_extraction.text import CountVectorizer
# # bq31_bow_encoder = CountVectorizer()

# # tmp = []
# # for i in zip(full_x[bq31_vars[0]], full_x[bq31_vars[1]], full_x[bq31_vars[2]]):
# #     tmp.append(" ".join(i))
# # bow = bq31_bow_encoder.fit_transform(tmp)
# # full_x = pd.concat([full_x, dataframe(bow.toarray(), columns=["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))])], axis=1)

# # tmp = []
# # for i in zip(test_x[bq31_vars[0]], test_x[bq31_vars[1]], test_x[bq31_vars[2]]):
# #     tmp.append(" ".join(i))
# # bow = bq31_bow_encoder.transform(tmp)
# # test_x = pd.concat([test_x, dataframe(bow.toarray(), columns=["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))])], axis=1)


# # transform to Label Encoding
# tmp = tmp.apply(lambda x: x if x in high_freq else "none")
# bq31_encoder = copy.deepcopy(MyLabelEncoder())
# bq31_encoder.fit_transform(tmp.to_frame(name="tmp"), ["tmp"])

# for i in range(max_size_tool):
#     full_x["bq31_" + str(i)] = bq31_encoder.transform(full_x["bq31_" + str(i)].to_frame(name="tmp"))
#     test_x["bq31_" + str(i)] = bq31_encoder.transform(test_x["bq31_" + str(i)].to_frame(name="tmp"))


# # "bq32" 이전 직업
# print(full_x["bq32"].value_counts())
# full_x["bq32"] = full_x["bq32"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq32"].value_counts().index[full_x["bq32"].value_counts() > i]) < 100:
#         high_freq = full_x["bq32"].value_counts().index[full_x["bq32"].value_counts() > i]
#         break
# # high_freq = full_x["bq32"].value_counts().index[full_x["bq32"].value_counts() > 2]
# print(len(high_freq))
# test_x["bq32"] = test_x["bq32"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq32"] = full_x["bq32"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq32"] = test_x["bq32"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq32"].value_counts())


# # "bq33" 전직가능직업
# print(full_x["bq33"].value_counts())
# full_x["bq33"] = full_x["bq33"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq33"].value_counts().index[full_x["bq33"].value_counts() > i]) < 100:
#         high_freq = full_x["bq33"].value_counts().index[full_x["bq33"].value_counts() > i]
#         break
# # high_freq = full_x["bq33"].value_counts().index[full_x["bq33"].value_counts() > 2]
# print(len(high_freq))
# test_x["bq33"] = test_x["bq33"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq33"] = full_x["bq33"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq33"] = test_x["bq33"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq33"].value_counts())


# # "bq34" 신직업 발굴
# full_x["bq34"] = full_x["bq34"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq34"].value_counts().index[full_x["bq34"].value_counts() > i]) < 100:
#         high_freq = full_x["bq34"].value_counts().index[full_x["bq34"].value_counts() > i]
#         break
# # high_freq = full_x["bq34"].value_counts().index[full_x["bq34"].value_counts() > 2]
# len(high_freq)
# test_x["bq34"] = test_x["bq34"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq34"] = full_x["bq34"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq34"] = test_x["bq34"].apply(lambda x: x if x in high_freq else "none")

# # "bq38_1" 전공
# print(full_x["bq38_1"].value_counts())
# full_x["bq38_1"] = full_x["bq38_1"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)
# high_freq = None
# for i in range(1, 100):
#     if len(full_x["bq38_1"].value_counts().index[full_x["bq38_1"].value_counts() > i]) < 100:
#         high_freq = full_x["bq38_1"].value_counts().index[full_x["bq38_1"].value_counts() > i]
#         break
# # high_freq = full_x["bq38_1"].value_counts().index[full_x["bq38_1"].value_counts() > 2]
# print(len(high_freq))
# test_x["bq38_1"] = test_x["bq38_1"].apply(lambda x: "none" if x == "없음" or x == "없다" or x == "모름" or x == "잘모르겠음" else x)

# # remove low frequency
# full_x["bq38_1"] = full_x["bq38_1"].apply(lambda x: x if x in high_freq else "none")
# test_x["bq38_1"] = test_x["bq38_1"].apply(lambda x: x if x in high_freq else "none")
# print(full_x["bq38_1"].value_counts())

# full_x.isna().sum()
# print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# test_x.isna().sum()
# print(test_x.isna().sum().index[full_x.isna().sum() != 0])




# # age transformation
# full_x["bq37_age_cat"] = pd.cut(full_x["bq37"], [i * 10 for i in range(1, 10)], right=False).astype("object")
# test_x["bq37_age_cat"] = pd.cut(test_x["bq37"], [i * 10 for i in range(1, 10)], right=False).astype("object")

# # === 불필요 컬럼 drop 및 이상치 제거 ===
# # 해당 업계 초봉 drop
# full_x.drop(["bq41_2"], axis=1, inplace=True)
# test_x.drop(["bq41_2"], axis=1, inplace=True)

# # # 불필요 텍스트 컬럼 drop
# # full_x.drop(["bq4_1b", "bq4_1c", "bq19_1", "bq31", "bq31_0", "bq31_1", "bq31_2"], axis=1, inplace=True)
# # test_x.drop(["bq4_1b", "bq4_1c", "bq19_1", "bq31", "bq31_0", "bq31_1", "bq31_2"], axis=1, inplace=True)
# # text_vars = diff(text_vars, ["bq4_1b", "bq4_1c", "bq19_1", "bq31", "bq31_0", "bq31_1", "bq31_2"])

# # 불필요 텍스트 컬럼 drop
# full_x.drop(["bq4_1b", "bq4_1c", "bq19_1", "bq31"], axis=1, inplace=True)
# test_x.drop(["bq4_1b", "bq4_1c", "bq19_1", "bq31"], axis=1, inplace=True)
# text_vars = diff(text_vars, ["bq4_1b", "bq4_1c", "bq19_1", "bq31"])

# # 이상치 aq41_1 == 33
# full_y = full_y[~(full_x["aq41_1"] == 33)]
# full_x = full_x[~(full_x["aq41_1"] == 33)]
# full_x.reset_index(drop=True, inplace=True)
# full_y.reset_index(drop=True, inplace=True)

# full_x["bq41_1"].fillna(0.0, inplace=True)
# full_x["bq41_3"].fillna(0.0, inplace=True)
# full_x["bq41_1_unemp"] = [1 if i == 0 else 0 for i in full_x["bq41_1"]]
# full_x["bq41_3_unemp"] = [1 if i == 0 else 0 for i in full_x["bq41_3"]]

# test_x["bq41_1"].fillna(0.0, inplace=True)
# test_x["bq41_3"].fillna(0.0, inplace=True)
# test_x["bq41_1_unemp"] = [1 if i == 0 else 0 for i in test_x["bq41_1"]]
# test_x["bq41_3_unemp"] = [1 if i == 0 else 0 for i in test_x["bq41_3"]]

# bin_vars = []
# ord_vars = list(full_x.columns[:82])
# # ord_vars = []
# # text_vars += ["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))]
# num_vars = ["bq23", "bq37", "bq41_1", "bq41_3"]


# full_x.isna().sum()
# print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# test_x.isna().sum()
# print(test_x.isna().sum().index[full_x.isna().sum() != 0])


# full_x[diff(full_x.columns, text_vars)] = full_x[diff(full_x.columns, text_vars)].fillna(0.0)
# test_x[diff(full_x.columns, text_vars)] = test_x[diff(full_x.columns, text_vars)].fillna(0.0)

# text_vars = ["bq31_" + str(i) for i in range(3)]
# cat_vars = diff(full_x.columns, num_vars + ord_vars + bin_vars + text_vars)

# # columns length check
# print("columns lenght check")
# print(len(full_x.columns) == len(num_vars) + len(ord_vars) + len(bin_vars) + len(cat_vars) + len(text_vars))



# # full_x.isna().sum()
# # print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# # test_x.isna().sum()
# # print(test_x.isna().sum().index[full_x.isna().sum() != 0])


# label_encoder = copy.deepcopy(MyLabelEncoder())
# full_x = label_encoder.fit_transform(full_x, cat_vars + bin_vars + ord_vars)
# for i in cat_vars:
#     print(full_x[i].value_counts().sort_index(), "\n")
# test_x = label_encoder.transform(test_x)
# for i in cat_vars:
#     print(test_x[i].value_counts().sort_index(), "\n")


# # string var check
# for i in full_x:
#     if full_x[i].dtype == "object":
#         print(i)

# # print(full_x.isna().sum())
# # print(full_x.isna().sum().sum())
# # print(test_x.isna().sum())
# # print(test_x.isna().sum().sum())

# knn_imputer = copy.deepcopy(MyKNNImputer())
# full_x = knn_imputer.fit_transform(full_x, full_y, cat_vars + bin_vars + ord_vars + text_vars)
# test_x = knn_imputer.transform(test_x)

# print(full_x.isna().sum().sum())
# print(test_x.isna().sum().sum())

# # string var check
# for i in full_x:
#     if full_x[i].dtype == "object":
#         print(i)

# # numerical to discrete integer
# # for mlp embeddin layer
# kbins_encoder = KBinsDiscretizer(10, encode="ordinal")
# full_x[num_vars] = kbins_encoder.fit_transform(full_x[num_vars])
# test_x[num_vars] = kbins_encoder.transform(test_x[num_vars])

# # === one hot encoding ===
# # print(full_x.isna().sum().sum())
# # print(test_x.isna().sum().sum())

# # print(full_x.shape[1] == len(cat_vars) + len(bin_vars) + len(ord_vars) + len(num_vars))

# # print(full_x.head(20))
# # print(test_x.head(20))

# # oh_encoder = copy.deepcopy(MyOneHotEncoder())
# # full_x_oh = oh_encoder.fit_transform(full_x, cat_vars + diff(text_vars, ["bq31_bow" + str(i) for i in range(len(bq31_bow_encoder.get_feature_names_out()))]))
# # test_x_oh = oh_encoder.transform(test_x)

# # print(full_x_oh.isna().sum().sum())
# # print(test_x_oh.isna().sum().sum())

# print(full_x.shape)
# print(test_x.shape)
# print(full_x_oh.shape)
# print(test_x_oh.shape)

before removing dups 9486
after removing dups 9486
before removing invalid values on target 9486
after removing invalid values on target 9104
{'knowcode': {'110101': 0, '110102': 1, '110103': 2, '110104': 3, '110105': 4, '110201': 5, '110202': 6, '110203': 7, '11101': 8, '11102': 9, '11201': 10, '12101': 11, '12102': 12, '121101': 13, '121102': 14, '121103': 15, '121104': 16, '121105': 17, '121201': 18, '12201': 19, '122101': 20, '122102': 21, '122103': 22, '122104': 23, '122105': 24, '122201': 25, '122301': 26, '12301': 27, '12401': 28, '12402': 29, '13101': 30, '131101': 31, '131201': 32, '131202': 33, '131203': 34, '132002': 35, '132004': 36, '13201': 37, '13202': 38, '13203': 39, '13204': 40, '13305': 41, '133101': 42, '133201': 43, '133202': 44, '133203': 45, '133204': 46, '133205': 47, '133301': 48, '133302': 49, '13401': 50, '134101': 51, '134102': 52, '134201': 53, '134301': 54, '134302': 55, '134303': 56, '134401': 57, '135001': 58, '13501': 59, '136001': 60, '136002': 61, '13

# Preprocessing - 2018

In [ ]:
full_x.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9072 entries, 0 to 9071
Data columns (total 139 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cq1      9072 non-null   int64  
 1   cq2      9072 non-null   int64  
 2   cq3      9072 non-null   int64  
 3   cq4      9072 non-null   int64  
 4   cq5      9072 non-null   int64  
 5   cq6      9072 non-null   int64  
 6   cq7      9072 non-null   int64  
 7   cq8      9072 non-null   int64  
 8   cq9      9072 non-null   int64  
 9   cq10     9072 non-null   int64  
 10  cq11     9072 non-null   int64  
 11  cq12     9072 non-null   int64  
 12  cq13     9072 non-null   int64  
 13  cq14     9072 non-null   int64  
 14  cq15     9072 non-null   int64  
 15  cq16     9072 non-null   int64  
 16  cq17     9072 non-null   int64  
 17  cq18     9072 non-null   float64
 18  cq19     9072 non-null   float64
 19  cq20     9072 non-null   float64
 20  cq21     9072 non-null   float64
 21  cq22     9072

In [ ]:
a = ["bq221", "bq222", "bq223", "bq231", "bq232", "bq233", "bq234",
     "bq235", "bq241", "bq242", "bq243", "bq244", "bq245"]

In [ ]:
# ===== Preprocessing =====
target_year = "2018"
target_var = "knowcode"
full_x = read_csv(folder_path + "train/KNOW_" + target_year + ".csv", dtype={target_var: str}, na_values=["", " "], encoding="utf-8")
full_x.drop(["idx"], axis=1, inplace=True)
print("before removing dups", full_x.shape[0])
full_x.drop_duplicates(inplace=True)
print("after removing dups", full_x.shape[0])

print("before removing invalid values on target", full_x.shape[0])
full_x[target_var].replace(" ", "", inplace=True)
full_x = full_x[(full_x[target_var] != "9999999") & (full_x[target_var] != nan)]
print("after removing invalid values on target", full_x.shape[0])
full_x.reset_index(drop=True, inplace=True)

full_y = full_x[target_var].to_frame()
target_encoder = copy.deepcopy(MyLabelEncoder())
full_y = target_encoder.fit_transform(full_y, [target_var])
print(target_encoder.dic_cat)
full_y.isna().sum()
full_y[target_var] = full_y[target_var].astype("int")
print(full_y[target_var].value_counts())
full_x.drop([target_var], axis=1, inplace=True)

test_x = read_csv(folder_path + "test/KNOW_" + target_year + "_test.csv", dtype={target_var: str}, na_values=["", " "], encoding="utf-8")
submission_idx_start = test_x["idx"].iloc[0]
submission_idx_end = test_x["idx"][-1:].iloc[0]
test_x.drop(["idx"], axis=1, inplace=True)



# # # text feature 정리
# # "bq4_1a" 필요자격증1
# # "bq4_1b" 필요자격증2
# # "bq4_1c" 필요자격증3
# # "bq5_2" 요구훈련종류
# # "bq19_1" : 향후 일자리 변화 이유
# # "bq30" : 유사직업명
# # "bq31" : 사용도구 및 프로그램
# # "bq32" : 이전직업
# # "bq33" : 전직 가능 직업
# # "bq34" : 신직업 발굴
# # "bq38_1" : 전공


# # # 자격증 컬럼 처리
# # 기술사 - 기능장 - 기사 - 산업기사 - 기능사
# remove_text = ["기술사", "기능장", "기사", "산업기사", "기능사",
#                "자격증", "자격", "면허증", "면허", "전문가"]

# text_vars = ["bq4_1b", "bq4_1c", "bq5_2", "bq19_1",
#              "bq30", "bq31", "bq32", "bq33", "bq34", "bq38_1"]

# # check low frequency word
# low_freq = full_x["bq4_1a"].value_counts().index[full_x["bq4_1a"].value_counts() < 2]

# full_x["bq4_1a"] = full_x["bq4_1a"].str.lower()
# for idx, value in enumerate(full_x["bq4_1a"]):
#     # break
#     # 띄어쓰기 제거
#     if isna(value) or value in low_freq:
#         full_x["bq4_1a"][idx] = "없음"
#     else:
#         value = value.replace(" ", "")
#         for i in remove_text:
#             value = value.replace(i, "")
#         value = re.sub('[1-9급()<>]', "", value)
#         full_x["bq4_1a"][idx] = value

# test_x["bq4_1a"] = test_x["bq4_1a"].str.lower()
# for idx, value in enumerate(test_x["bq4_1a"]):
#     # break
#     # 띄어쓰기 제거
#     if isna(value) or value in low_freq:
#         test_x["bq4_1a"][idx] = "없음"
#     else:
#         value = value.replace(" ", "")
#         for i in remove_text:
#             value = value.replace(i, "")
#         value = re.sub('[1-9급()<>]', "", value)
#         test_x["bq4_1a"][idx] = value


# full_x.drop(text_vars, axis=1, inplace=True)
# test_x.drop(text_vars, axis=1, inplace=True)



# full_x.isna().sum()
# print(full_x.isna().sum().index[full_x.isna().sum() != 0])
# test_x.isna().sum()
# print(test_x.isna().sum().index[full_x.isna().sum() != 0])

# # age transformation
# full_x["bq37"] = pd.cut(full_x["bq37"], [i * 10 for i in range(1, 10)]).astype("object")
# test_x["bq37"] = pd.cut(test_x["bq37"], [i * 10 for i in range(1, 10)]).astype("object")

# # 이상치 aq41_1 == 33
# full_y = full_y[~(full_x["aq41_1"] == 33)]
# full_x = full_x[~(full_x["aq41_1"] == 33)]

# # # 해당 업계 초봉, 자격증 컬럼 drop
# # full_x.drop(["bq4_1a", "bq41_2"], axis=1, inplace=True)
# # test_x.drop(["bq4_1a", "bq41_2"], axis=1, inplace=True)

# # 해당 업계 초봉 drop
# full_x.drop(["bq41_2"], axis=1, inplace=True)
# test_x.drop(["bq41_2"], axis=1, inplace=True)

# bin_vars = []
# ord_vars = list(full_x.columns[:82])
# num_vars = ["bq23", "bq41_1", "bq41_3"]
# cat_vars = diff(full_x.columns, num_vars + ord_vars + bin_vars)

# for i in cat_vars:
#     try:
#         full_x[i] = full_x[i].fillna(0.0).astype("int")
#         test_x[i] = test_x[i].fillna(0.0).astype("int")
#     except:
#         pass

# label_encoder = copy.deepcopy(MyLabelEncoder())
# full_x = label_encoder.fit_transform(full_x, cat_vars)
# for i in cat_vars:
#     print(full_x[i].value_counts().sort_index(), "\n")
# test_x = label_encoder.transform(test_x)
# for i in cat_vars:
#     print(test_x[i].value_counts().sort_index(), "\n")

# print(full_x.isna().sum())
# print(test_x.isna().sum())

# knn_imputer = copy.deepcopy(MyKNNImputer())
# full_x = knn_imputer.fit_transform(full_x, full_y, cat_vars + ord_vars)
# test_x = knn_imputer.transform(test_x)

# print(full_x.isna().sum().sum())
# print(test_x.isna().sum().sum())

# print(full_x.shape[1] == len(cat_vars) + len(bin_vars) + len(ord_vars) + len(num_vars))

# print(full_x.head(20))
# print(test_x.head(20))

# oh_encoder = MyOneHotEncoder()
# full_x_oh = oh_encoder.fit_transform(full_x, cat_vars)
# test_x_oh = oh_encoder.transform(test_x)

# print(full_x_oh.isna().sum().sum())
# print(test_x_oh.isna().sum().sum())

# print(full_x.shape)
# print(test_x.shape)
# print(full_x_oh.shape)
# print(test_x_oh.shape)

before removing dups 9072
after removing dups 9072
before removing invalid values on target 9072
after removing invalid values on target 9072
{'knowcode': {'110101': 0, '110102': 1, '110103': 2, '110104': 3, '110105': 4, '110201': 5, '110202': 6, '110203': 7, '11101': 8, '11102': 9, '11201': 10, '12101': 11, '12102': 12, '121101': 13, '121102': 14, '121103': 15, '121104': 16, '121105': 17, '121201': 18, '12201': 19, '122101': 20, '122102': 21, '122103': 22, '122104': 23, '122105': 24, '122201': 25, '122301': 26, '12301': 27, '12401': 28, '12402': 29, '13101': 30, '131101': 31, '131201': 32, '131202': 33, '131203': 34, '132002': 35, '132004': 36, '13201': 37, '13202': 38, '13203': 39, '13204': 40, '13305': 41, '133101': 42, '133201': 43, '133202': 44, '133203': 45, '133204': 46, '133205': 47, '133301': 48, '133302': 49, '133901': 50, '13401': 51, '134101': 52, '134102': 53, '134103': 54, '134201': 55, '134301': 56, '134302': 57, '134303': 58, '134401': 59, '135001': 60, '13501': 61, '13

# Modeling

In [ ]:
# ===== modeling =====
from datetime import timezone
timezone_kst = timezone(timedelta(hours=9))

# <10-folds inference>

# fold spliter setting
kfolds_spliter = StratifiedKFold(10, random_state=1234, shuffle=True)
# cut_off = 0.5

# result container setting
val_prob = np.zeros((full_x.shape[0], len(full_y.value_counts())))
test_prob = np.zeros((test_x.shape[0], len(full_y.value_counts())))
val_perf = []

# optuna function
def optuna_objective_function(trial: Trial, train_x, train_y, val_x, val_y, model_name, ntrees=None, eta=None):
    if model_name == "CAT_GBM":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, step=2),  
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }

        model = cat.CatBoostClassifier(boosting_type="Plain", loss_function="MultiClass", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    elif model_name == "CAT_ORD":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, step=2),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }

        model = cat.CatBoostClassifier(boosting_type="Ordered", loss_function="MultiClass", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    elif model_name == "XGB_GBT":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "subsample": trial.suggest_float("subsample", 0.5, 0.95, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.95, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, step=0.01),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.01, 10.0, step=0.01),
            "gamma": trial.suggest_float("gamma", 0.0, 10.0, step=0.01),
            "max_delta_step":  trial.suggest_float("max_delta_step", 0.0, 10.0, step=0.01)
            # "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }
        model = xgb.XGBClassifier(booster="gbtree", objective="multi:softmax",
                                tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
                                n_estimators=int(ntrees * 0.2),
                                learning_rate=eta / 10,
                                random_state=fold, verbosity=0, use_label_encoder=False,
                                **tuning_params)     
        model.fit(train_x, train_y, verbose=False)
    # default randomforest
    else:
        tuning_params = {
            "num_leaves": trial.suggest_categorical("num_leaves", [pow(2, i) - 1 for i in [4, 5, 6, 7, 8]]),
            "subsample": trial.suggest_float("subsample", 0.5, 0.9, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.9, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 50, step=5),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-4, 1e-2),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 2.0, step=0.01)
        }
        model = lgb.LGBMClassifier(boosting_type="rf", objective="binary", force_row_wise=True,
                                   n_estimators=int(ntrees * 0.2), subsample_freq=1, device_type="gpu", gpu_device_id=0,
                                   n_jobs=cpu_count(), random_state=fold, **tuning_params)
        model.fit(train_x, train_y, categorical_feature=categoIdx,
                  callbacks=[lgb.callback.log_evaluation(period=np.inf, show_stdv=False)])

    prob = model.predict_proba(val_x)
    pred = prob.argmax(axis=1)
    opt_score = metrics.f1_score(val_y, pred, average="macro")

    trial.report(opt_score, step=trial.number)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return opt_score

In [ ]:
from tensorflow import losses as tf_loss
hiddenLayers = 256
dropoutRate = 1 / 2**2

def createNN_LP_V1(nCols, n_classes): 
    # === input layers ===
    B0_input = layers.Input(shape=nCols, dtype="float32", name="B0_input")
    B0_embedding = layers.Embedding(input_dim=1024, output_dim=8, name="B0_embedding")(B0_input)
    B0_flatten = layers.Flatten()(B0_embedding)
    # B0_dense = layers.Dense(hiddenLayers * 2, kernel_regularizer="l1")(B0_flatten)
    B0_dropout = layers.Dropout(0.5)(B0_flatten)

    # === learning layers ===
    B1_dense = tfa.layers.WeightNormalization(
        layers.Dense(
            units=hiddenLayers, activation="selu"), name="B1_dense"
    )(B0_dropout)
    B1_dropout = layers.Dropout(rate=dropoutRate, name="B1_dropout")(B1_dense)
    B1_concat = layers.Concatenate(name="B1_concat")([B0_dropout, B1_dropout])

    B2_dense = tfa.layers.WeightNormalization(
        layers.Dense(
            units=hiddenLayers, activation="relu"), name="B2_dense"
    )(B1_concat)
    B2_dropout = layers.Dropout(rate=dropoutRate, name="B2_dropout")(B2_dense)
    B2_concat = layers.Concatenate(name="B2_concat")([B0_dropout, B1_dropout, B2_dropout])

    B3_dense = tfa.layers.WeightNormalization(
        layers.Dense(
            units=hiddenLayers, activation="elu"), name="B3_dense"
    )(B2_concat)

    layer_classifier = layers.Dense(n_classes, activation="softmax", name="classifier")(B3_dense)

    return Model(B0_input, layer_classifier)

# creating the dataset
def create_dataset(x=None, y=None, batch_size=None,shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(1024, reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# XGBoost try1

In [ ]:
# ===== modeling =====
model_name = "XGBoost_Gbtree_Try1"
ntrees = 5000
eta = 1e-2
def xgb_f1_score(predt: np.ndarray, dtrain: xgb.DMatrix):
    return 'F1', -metrics.f1_score(dtrain.get_label(), predt.argmax(axis=1), average="macro")

total_time = time()
seed_everything()
for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

    fold_time = time()
    print("===== Fold", fold+1, "Prediction =====\n")
    fold_dir = "fold_" + str(fold)
    wnb_init_info = {"project": "dacon_jobknow_recommendation",
                     "group": model_name,
                     "name": fold_dir + "_optuna"}

    print("<XGBoost>")

    
    optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
                                    wandb_kwargs=wnb_init_info)]
    wandb.config.step = 0

    train_ds_x = full_x_oh.iloc[nonkIdx].reset_index(drop=True).copy()
    train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
    val_ds_x = full_x_oh.iloc[kIdx].reset_index(drop=True).copy()
    val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
    test_ds_x = test_x_oh.copy()
    
    optuna_timout = int(12 * 3600 / kfolds_spliter.get_n_splits())
    # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
    optuna_study = create_study(direction='maximize', sampler=TPESampler())
    optuna_study.optimize(lambda trial: optuna_objective_function(
        trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
                          n_jobs=1, n_trials=300, timeout=optuna_timout, callbacks=optuna_cb)
    
    # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
    # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

    model = xgb.XGBClassifier(booster="gbtree", objective="multi:softmax",
                            tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
                            n_estimators=ntrees,
                            learning_rate=eta,
                            random_state=fold, verbosity=0, use_label_encoder=False,
                            **optuna_study.best_params)

    model.fit(train_ds_x, train_ds_y, eval_metric=xgb_f1_score, verbose=False,
              eval_set=[(val_ds_x, val_ds_y)], early_stopping_rounds=int(ntrees * 0.2))

    # model.fit(train_ds_x, train_ds_y, verbose=False)
    
    print("tuned params --->", optuna_study.best_params)
    print("best trees --->", model.best_iteration, "\n")

    prob = model.predict_proba(val_ds_x)
    pred = prob.argmax(axis=1)
    val_perf.append(metrics.f1_score(val_ds_y, pred, average="macro"))
    val_prob[kIdx] = prob
    
    print("Fold running time --->", np.round(time() - fold_time, 3))
    print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
    print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
    # cm = wandb.plot.confusion_matrix(
    #     y_true=list(val_ds_y),
    #     preds=pred,
    #     class_names=[0, 1])
    wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration}, commit=True)
    
    prob = model.predict_proba(test_ds_x)
    test_prob += prob / kfolds_spliter.get_n_splits()
    wandb.finish()

In [ ]:
# wandb.finish()

In [ ]:
# # ===== modeling =====

# model_name = "XGBoost_Gbtree_Try2"
# ntrees = 5000
# eta = 1e-2
# def xgb_f1_score(predt: np.ndarray, dtrain: xgb.DMatrix):
#     return 'F1', -metrics.f1_score(dtrain.get_label(), predt.argmax(axis=1), average="macro")

# total_time = time()
# seed_everything()
# for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

#     fold_time = time()
#     print("===== Fold", fold+1, "Prediction =====\n")
#     fold_dir = "fold_" + str(fold)
#     # wnb_init_info = {"project": "dacon_jobknow_recommendation",
#     #                  "group": model_name,
#     #                  "name": fold_dir + "_optuna"}

#     print("<XGBoost>")

#     wandb.init(project="dacon_jobknow_recommendation",
#                group=model_name,
#                name=fold_dir)
#     # optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
#     #                                 wandb_kwargs=wnb_init_info)]
#     wandb.config.step = 0

#     train_ds_x = full_x_oh.iloc[nonkIdx].reset_index(drop=True).copy()
#     train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
#     val_ds_x = full_x_oh.iloc[kIdx].reset_index(drop=True).copy()
#     val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
#     test_ds_x = test_x_oh.copy()
    
#     # optuna_timout = int(18 * 3600 / kfolds_spliter.get_n_splits())
#     # # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
#     # optuna_study = create_study(direction='maximize', sampler=TPESampler())
#     # optuna_study.optimize(lambda trial: optuna_objective_function(
#     #     trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
#     #                       n_jobs=1, n_trials=10, timeout=optuna_timout, callbacks=optuna_cb)
    
#     # break
    
#     # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
#     # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

#     model = xgb.XGBClassifier(booster="gbtree", objective="multi:softmax",
#                             tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
#                             n_estimators=ntrees,
#                             learning_rate=eta,
#                             subsample=0.8, colsample_bytree=0.8,
#                             random_state=fold, verbosity=0, use_label_encoder=False)

#     model.fit(train_ds_x, train_ds_y, eval_metric=xgb_f1_score, verbose=False,
#               eval_set=[(val_ds_x, val_ds_y)], early_stopping_rounds=int(ntrees * 0.2))

#     # model.fit(train_ds_x, train_ds_y, verbose=False)
    
#     # print("tuned params --->", optuna_study.best_params)
#     print("best trees --->", model.best_iteration, "\n")

#     prob = model.predict_proba(val_ds_x)
#     pred = prob.argmax(axis=1)
#     val_perf.append(metrics.f1_score(val_ds_y, pred, average="macro"))
#     val_prob[kIdx] = prob
    
#     print("Fold running time --->", np.round(time() - fold_time, 3))
#     print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
#     print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
#     # cm = wandb.plot.confusion_matrix(
#     #     y_true=list(val_ds_y),
#     #     preds=pred,
#     #     class_names=[0, 1])
#     # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
#     wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration}, commit=True)
    
#     prob = model.predict_proba(test_ds_x)
#     test_prob += prob / kfolds_spliter.get_n_splits()
#     wandb.finish()

# XGBoost try2 (no tuning) - more text prep

In [ ]:
# # ===== modeling =====
# model_name = "XGBoost_Gbtree_Try2"
# ntrees = 5000
# eta = 1e-2
# def xgb_f1_score(predt: np.ndarray, dtrain: xgb.DMatrix):
#     return 'F1', -metrics.f1_score(dtrain.get_label(), predt.argmax(axis=1), average="macro")

# total_time = time()
# seed_everything()
# for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

#     fold_time = time()
#     print("===== Fold", fold+1, "Prediction =====\n")
#     fold_dir = "fold_" + str(fold)
#     # wnb_init_info = {"project": "dacon_jobknow_recommendation",
#     #                  "group": model_name,
#     #                  "name": fold_dir + "_optuna"}

#     print("<XGBoost>")

#     wandb.init(project="dacon_jobknow_recommendation",
#                group=model_name,
#                name=fold_dir)
#     # optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
#     #                                 wandb_kwargs=wnb_init_info)]
#     wandb.config.step = 0

#     train_ds_x = full_x_oh.iloc[nonkIdx].reset_index(drop=True).copy()
#     train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
#     val_ds_x = full_x_oh.iloc[kIdx].reset_index(drop=True).copy()
#     val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
#     test_ds_x = test_x_oh.copy()
    
#     # optuna_timout = int(18 * 3600 / kfolds_spliter.get_n_splits())
#     # # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
#     # optuna_study = create_study(direction='maximize', sampler=TPESampler())
#     # optuna_study.optimize(lambda trial: optuna_objective_function(
#     #     trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
#     #                       n_jobs=1, n_trials=10, timeout=optuna_timout, callbacks=optuna_cb)
    
#     # break
    
#     # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
#     # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

#     model = xgb.XGBClassifier(booster="gbtree", objective="multi:softmax",
#                             tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
#                             n_estimators=ntrees,
#                             learning_rate=eta,
#                             subsample=0.8, colsample_bytree=0.8,
#                             random_state=fold, verbosity=0, use_label_encoder=False)

#     model.fit(train_ds_x, train_ds_y, eval_metric=xgb_f1_score, verbose=False,
#               eval_set=[(val_ds_x, val_ds_y)], early_stopping_rounds=int(ntrees * 0.2))

#     # model.fit(train_ds_x, train_ds_y, verbose=False)
    
#     # print("tuned params --->", optuna_study.best_params)
#     print("best trees --->", model.best_iteration, "\n")

#     prob = model.predict_proba(val_ds_x)
#     pred = prob.argmax(axis=1)
#     val_perf.append(metrics.f1_score(val_ds_y, pred, average="macro"))
#     val_prob[kIdx] = prob
    
#     print("Fold running time --->", np.round(time() - fold_time, 3))
#     print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
#     print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
#     # cm = wandb.plot.confusion_matrix(
#     #     y_true=list(val_ds_y),
#     #     preds=pred,
#     #     class_names=[0, 1])
#     # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
#     wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration}, commit=True)
    
#     prob = model.predict_proba(test_ds_x)
#     test_prob += prob / kfolds_spliter.get_n_splits()
#     wandb.finish()

# XGBoost try3 (no tuning) - more text prep

In [ ]:
# # ===== modeling =====
# model_name = "XGBoost_Gbtree_Try3"
# ntrees = 5000
# eta = 1e-2
# def xgb_f1_score(predt: np.ndarray, dtrain: xgb.DMatrix):
#     return 'F1', -metrics.f1_score(dtrain.get_label(), predt.argmax(axis=1), average="macro")

# total_time = time()
# seed_everything()
# for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

#     fold_time = time()
#     print("===== Fold", fold+1, "Prediction =====\n")
#     print("Start Time :", datetime.utcnow().astimezone(timezone_kst))
#     fold_dir = "fold_" + str(fold)
#     # wnb_init_info = {"project": "dacon_jobknow_recommendation",
#     #                  "group": model_name,
#     #                  "name": fold_dir + "_optuna"}

#     print("<XGBoost>")


#     wandb.init(project="dacon_jobknow_recommendation",
#                group=model_name,
#                name=fold_dir)
#     # optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
#     #                                 wandb_kwargs=wnb_init_info)]
#     wandb.config.step = 0

#     train_ds_x = full_x_oh.iloc[nonkIdx].reset_index(drop=True).copy()
#     train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
#     val_ds_x = full_x_oh.iloc[kIdx].reset_index(drop=True).copy()
#     val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
#     test_ds_x = test_x_oh.copy()
    
#     # optuna_timout = int(18 * 3600 / kfolds_spliter.get_n_splits())
#     # # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
#     # optuna_study = create_study(direction='maximize', sampler=TPESampler())
#     # optuna_study.optimize(lambda trial: optuna_objective_function(
#     #     trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
#     #                       n_jobs=1, n_trials=10, timeout=optuna_timout, callbacks=optuna_cb)
    
#     # break
    
#     # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
#     # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

#     model = xgb.XGBClassifier(booster="gbtree", objective="multi:softmax",
#                             tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
#                             n_estimators=ntrees,
#                             learning_rate=eta,
#                             subsample=0.8, colsample_bynode=0.5,
#                             random_state=fold, verbosity=0, use_label_encoder=False)

#     model.fit(train_ds_x, train_ds_y, eval_metric=xgb_f1_score, verbose=False,
#               eval_set=[(val_ds_x, val_ds_y)], early_stopping_rounds=int(ntrees * 0.2))

#     # model.fit(train_ds_x, train_ds_y, verbose=False)
    
#     # print("tuned params --->", optuna_study.best_params)
#     print("best trees --->", model.best_iteration, "\n")

#     prob = model.predict_proba(val_ds_x)
#     pred = prob.argmax(axis=1)
#     val_perf.append(metrics.f1_score(val_ds_y, pred, average="macro"))
#     val_prob[kIdx] = prob
    
#     print("Fold running time --->", np.round(time() - fold_time, 3))
#     print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
#     print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
#     # cm = wandb.plot.confusion_matrix(
#     #     y_true=list(val_ds_y),
#     #     preds=pred,
#     #     class_names=[0, 1])
#     # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
#     wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration}, commit=True)
    
#     prob = model.predict_proba(test_ds_x)
#     test_prob += prob / kfolds_spliter.get_n_splits()
#     wandb.finish()

# XGBoost try4 (no tuning) - more text prep

In [ ]:
# # ===== modeling =====
# model_name = "XGBoost_Gbtree_Try4"
# ntrees = 5000
# eta = 1e-2
# def xgb_f1_score(predt: np.ndarray, dtrain: xgb.DMatrix):
#     return 'F1', -metrics.f1_score(dtrain.get_label(), predt.argmax(axis=1), average="macro")

# total_time = time()
# seed_everything()
# for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

#     fold_time = time()
#     print("===== Fold", fold+1, "Prediction =====\n")
#     print("Start Time :", datetime.utcnow().astimezone(timezone_kst))
#     fold_dir = "fold_" + str(fold)
#     # wnb_init_info = {"project": "dacon_jobknow_recommendation",
#     #                  "group": model_name,
#     #                  "name": fold_dir + "_optuna"}

#     print("<XGBoost>")


#     wandb.init(project="dacon_jobknow_recommendation",
#                group=model_name,
#                name=fold_dir)
#     # optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
#     #                                 wandb_kwargs=wnb_init_info)]
#     wandb.config.step = 0

#     train_ds_x = full_x_oh.iloc[nonkIdx].reset_index(drop=True).copy()
#     train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
#     val_ds_x = full_x_oh.iloc[kIdx].reset_index(drop=True).copy()
#     val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
#     test_ds_x = test_x_oh.copy()
    
#     # optuna_timout = int(18 * 3600 / kfolds_spliter.get_n_splits())
#     # # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
#     # optuna_study = create_study(direction='maximize', sampler=TPESampler())
#     # optuna_study.optimize(lambda trial: optuna_objective_function(
#     #     trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
#     #                       n_jobs=1, n_trials=10, timeout=optuna_timout, callbacks=optuna_cb)
    
#     # break
    
#     # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
#     # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

#     model = xgb.XGBClassifier(booster="gbtree", objective="multi:softmax",
#                             tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
#                             n_estimators=ntrees,
#                             learning_rate=eta,
#                             subsample=0.8, colsample_bynode=0.8,
#                             random_state=fold, verbosity=0, use_label_encoder=False)

#     model.fit(train_ds_x, train_ds_y, eval_metric=xgb_f1_score, verbose=False,
#               eval_set=[(val_ds_x, val_ds_y)], early_stopping_rounds=int(ntrees * 0.2))

#     # model.fit(train_ds_x, train_ds_y, verbose=False)
    
#     # print("tuned params --->", optuna_study.best_params)
#     print("best trees --->", model.best_iteration, "\n")

#     prob = model.predict_proba(val_ds_x)
#     pred = prob.argmax(axis=1)
#     val_perf.append(metrics.f1_score(val_ds_y, pred, average="macro"))
#     val_prob[kIdx] = prob
    
#     print("Fold running time --->", np.round(time() - fold_time, 3))
#     print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
#     print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
#     # cm = wandb.plot.confusion_matrix(
#     #     y_true=list(val_ds_y),
#     #     preds=pred,
#     #     class_names=[0, 1])
#     # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
#     wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration}, commit=True)
    
#     prob = model.predict_proba(test_ds_x)
#     test_prob += prob / kfolds_spliter.get_n_splits()
#     wandb.finish()

# CatBoost - GBM try1 (no tuning)

In [ ]:
# # ===== modeling =====
# model_name = "CatBoost_GBM_Try1"
# total_time = time()
# seed_everything()
# for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

#     fold_time = time()
#     print("===== Fold", fold+1, "Prediction =====\n")
#     fold_dir = "fold_" + str(fold)
#     # wnb_init_info = {"project": "dacon_jobknow_recommendation",
#     #                  "group": model_name,
#     #                  "name": fold_dir + "_optuna"}

#     print("<CatBoost - GBM>")
#     ntrees = 5000
#     eta = 1e-2
    
#     wandb.init(project="dacon_jobknow_recommendation",
#                group=model_name,
#                name=fold_dir)
#     # optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
#     #                                 wandb_kwargs=wnb_init_info)]
#     wandb.config.step = 0

#     train_ds_x = full_x.iloc[nonkIdx].reset_index(drop=True).copy()
#     train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
#     val_ds_x = full_x.iloc[kIdx].reset_index(drop=True).copy()
#     val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
#     test_ds_x = test_x.copy()
    
#     # optuna_timout = int(18 * 3600 / kfolds_spliter.get_n_splits())
#     # # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
#     # optuna_study = create_study(direction='maximize', sampler=TPESampler())
#     # optuna_study.optimize(lambda trial: optuna_objective_function(
#     #     trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
#     #                       n_jobs=1, n_trials=10, timeout=optuna_timout, callbacks=optuna_cb)
    
#     # break
    
#     # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
#     # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

#     model = cat.CatBoostClassifier(boosting_type="Plain", loss_function="MultiClass", task_type="GPU",
#                                 n_estimators=ntrees,
#                                 learning_rate=eta,       
#                                 one_hot_max_size=3, leaf_estimation_method="Gradient",
#                                 eval_metric="TotalF1:average=Macro",
#                                 # leaf_estimation_iterations=5,
#                                 # max_ctr_complexity=2,
#                                 logging_level="Silent", random_state=fold, thread_count=cpu_count())

#     model.fit(train_ds_x, train_ds_y, cat_features=categoIdx,
#               eval_set=[(val_ds_x, val_ds_y)], use_best_model=True,
#               early_stopping_rounds=int(ntrees * 0.2))

#     # model.fit(train_ds_x, train_ds_y, verbose=False)
    
#     # print("tuned params --->", optuna_study.best_params)
#     print("best trees --->", model.best_iteration_, "\n")

#     prob = model.predict_proba(val_ds_x)
#     pred = prob.argmax(axis=1)
#     val_perf.append(metrics.f1_score(val_ds_y, pred, average="macro"))
#     val_prob[kIdx] = prob
    
#     print("Fold running time --->", np.round(time() - fold_time, 3))
#     print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
#     print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
#     # cm = wandb.plot.confusion_matrix(
#     #     y_true=list(val_ds_y),
#     #     preds=pred,
#     #     class_names=[0, 1])
#     # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
#     wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration_}, commit=True)
    
#     prob = model.predict_proba(test_ds_x)
#     test_prob += prob / kfolds_spliter.get_n_splits()
#     wandb.finish()

In [ ]:
# wandb.finish()

# MLP_LP_V1 try1

In [ ]:
# ===== modeling =====
model_name = "MLP_LP_Try2"
epochs = 100
patient_epochs = 20
batch_size = 16
eta = 1e-3
weight_decay = 1e-4
model_save_flag = False
checkpoint_filepath = folder_path + 'models/tmp_checkpoint/'
target_onehot = copy.deepcopy(MyOneHotEncoder())
target_onehot.fit_transform(full_y.copy(), target_var)

total_time = time()
seed_everything()
for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

    fold_time = time()
    print("===== Fold", fold+1, "Prediction =====\n")
    tf.keras.backend.clear_session()
    fold_dir = "fold_" + str(fold)
    log_dir = folder_path + "models/" + model_name + "/logs/" + fold_dir
    if os.path.isdir(log_dir): shutil.rmtree(log_dir)
    createFolder(log_dir)
    
    tensorboard_callback = tf_callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    try:
        wandb.tensorboard.patch(root_logdir=log_dir)
    except:
        pass
    
    wandb.init(
        project="dacon_jobknow_recommendation",
        group=model_name,
        name=fold_dir
    ); wandb.config.step = 0

    print("<MLP - LP>")

    train_ds_x = full_x.iloc[nonkIdx].reset_index(drop=True).copy()
    train_ds_y = target_onehot.transform(full_y.iloc[nonkIdx].reset_index(drop=True).copy())
    train_ds = create_dataset(train_ds_x, train_ds_y, batch_size, True)
    val_ds_x = full_x.iloc[kIdx].reset_index(drop=True).copy()
    val_ds_y = target_onehot.transform(full_y.iloc[kIdx].reset_index(drop=True).copy())
    val_ds = create_dataset(val_ds_x, val_ds_y, batch_size, False)
    test_ds_x = test_x.copy()
    test_ds = create_dataset(test_ds_x, None, batch_size, False)

    cb_reduceLR = tf_callbacks.ReduceLROnPlateau(patience=1, factor=0.6, min_lr=1e-7)
    cb_earlyStopping = tf_callbacks.EarlyStopping(patience=patient_epochs, monitor='val_f1', mode='max')
    cb_modelsave = tf_callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_f1', mode='max',
                                                save_weights_only=True, save_best_only=True)

    model = createNN_LP_V1(train_ds_x.shape[1], len(full_y[target_var].unique()))
    model.compile(
        optimizer=tfa.optimizers.AdamW(learning_rate=eta, weight_decay=weight_decay),
        loss=tf_loss.CategoricalCrossentropy(),
        metrics=tfa.metrics.F1Score(num_classes=len(full_y[target_var].unique()), average="macro", threshold=None, name="f1")
    )
    model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=0,
            callbacks=[cb_reduceLR, cb_earlyStopping, cb_modelsave, tensorboard_callback, TqdmCallback(verbose=0)])
    model.load_weights(checkpoint_filepath)
    
    # print("tuned params --->", optuna_study.best_params)
    # print("best trees --->", model.best_iteration_, "\n")

    prob = model.predict(val_ds_x)
    pred = prob.argmax(axis=1)
    val_perf.append(metrics.f1_score(full_y.iloc[kIdx].reset_index(drop=True).copy(), pred, average="macro"))
    val_prob[kIdx] = prob
    
    print("Fold running time --->", np.round(time() - fold_time, 3))
    print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
    print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
    # cm = wandb.plot.confusion_matrix(
    #     y_true=list(val_ds_y),
    #     preds=pred,
    #     class_names=[0, 1])
    # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
    # wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration_}, commit=True)
    wandb.log(data={"fold": fold, "f1_final": val_perf[-1]}, commit=True)
    
    prob = model.predict(test_ds_x)
    test_prob += prob / kfolds_spliter.get_n_splits()
    wandb.finish()

===== Fold 1 Prediction =====



<MLP - LP>


0epoch [00:00, ?epoch/s]

Fold running time ---> 139.75
MLP_LP_Try2 Fold 1 F1 Score ---> 0.4512723671941549
MLP_LP_Try2 Average F1 Score ---> 0.4512723671941549 




f1_final,▁
fold,▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/epoch_f1,▁▂▃▄▅▅▆▇▇▇▇█████████████████▇▇▇▇▇▇▇▇▇▇
train/epoch_loss,█▅▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▄▄▅▅▆▆▇▇▇███
train/epoch_lr,██████▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
validation/epoch_f1,▁▃▅▆▆▆▇▇▇▇▇████████████████████████▇▇▇
validation/epoch_loss,▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇▇███
validation/evaluation_f1_vs_iterations,▁▃▅▆▆▆▇▇▇▇▇████████████████████████▇▇▇
validation/evaluation_loss_vs_iterations,▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇▇███


...

0epoch [00:00, ?epoch/s]

wandb: WARNING Found logdirectory outside of given root_logdir, dropping given root_logdir for eventfile in /content/drive/MyDrive/Colab Notebooks/projects/Dacon/jobknow_recommendation/models/MLP_LP_Try2/logs/fold_9/train


wandb: WARNING Found logdirectory outside of given root_logdir, dropping given root_logdir for eventfile in /content/drive/MyDrive/Colab Notebooks/projects/Dacon/jobknow_recommendation/models/MLP_LP_Try2/logs/fold_9/validation


Fold running time ---> 154.423
MLP_LP_Try2 Fold 10 F1 Score ---> 0.45404319998556986
MLP_LP_Try2 Average F1 Score ---> 0.4609563115262336 




f1_final,▁
fold,▁
global_step,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train/epoch_f1,▁▂▃▄▅▅▆▆▇▇▇█████████████████▇▇▇▇▇▇▇▇▇▇▇▇
train/epoch_loss,▇▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇▇████
train/epoch_lr,███████▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
validation/epoch_f1,▁▃▅▆▆▆▆▇▇▇█████████████████████▇▇▇▇▇▇▇▇▇
validation/epoch_loss,▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▆▆▆▇▇▇▇███
validation/evaluation_f1_vs_iterations,▁▃▅▆▆▆▆▇▇▇█████████████████████▇▇▇▇▇▇▇▇▇
validation/evaluation_loss_vs_iterations,▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▆▆▆▇▇▇▇███


In [ ]:
# wandb.finish()

# Submission

In [ ]:
print(model_name)
createFolder(folder_path + "models/" + model_name + "/")
dataframe(val_prob).to_csv(folder_path + "models/" + model_name + "/val_prob_" + target_year + ".csv", index=False)
dataframe(test_prob).to_csv(folder_path + "models/" + model_name + "/test_prob_" + target_year + ".csv", index=False)
submission = read_csv(folder_path + "sample_submission.csv")
submission[target_var][submission_idx_start:submission_idx_end+1] = [list(target_encoder.dic_cat["knowcode"].keys())[list(target_encoder.dic_cat["knowcode"].values()).index(i)] for i in test_prob.argmax(axis=1)]
submission.to_csv(folder_path + "models/" + model_name + "/submission_" + model_name + "_" + target_year + ".csv", index=False)
submission.head(10)

MLP_LP_Try2


,idx,knowcode
0,0,29401
1,1,155203
2,2,416101
3,3,121102
4,4,412003
5,5,414503
6,6,232101
7,7,12102
8,8,212101
9,9,841101
